In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
# Cargar el archivo (ajusta el nomre del archivo)
df = pd.read_excel(r"BROCOLI CONGELADO 2023.xlsx")

In [4]:
entradallm = df[["FECHA DESPACHO","KGS NETO", "PAIS DESTINO","FOB U$S", "ITEM"]]
entradallm["KGS_TOTAL"]= entradallm["KGS NETO"]*entradallm["ITEM"]
entradallm = entradallm.drop(["KGS NETO", "ITEM"],axis=1)

entradallm = entradallm.sort_values("FECHA DESPACHO")

entradallm["AÑO"] = entradallm["FECHA DESPACHO"].dt.year
entradallm["SEMANA"] = entradallm["FECHA DESPACHO"].dt.isocalendar().week
entradallm = entradallm.drop("FECHA DESPACHO",axis=1)

entradallm = entradallm.groupby(["AÑO", "SEMANA", "PAIS DESTINO"]).sum().reset_index()

entradallm = entradallm.sort_values("SEMANA",ascending=True)
entradallm = entradallm

C:\Users\DIEGO\AppData\Local\Temp\ipykernel_4064\2226708941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entradallm["KGS_TOTAL"]= entradallm["KGS NETO"]*entradallm["ITEM"]


In [5]:
entradallm["US/KG"] = entradallm["FOB U$S"]/entradallm["KGS_TOTAL"]

In [6]:
def eval_pais(pais, fun_predict):
    country = entradallm.loc[entradallm["PAIS DESTINO"] == pais]
    data_predict = country.loc[entradallm["SEMANA"] == 52]
    if data_predict.empty:
       # print("No hay datos para el país")
        return None
    data_past = country.loc[entradallm["SEMANA"] < 52]
    data_predict = data_predict["US/KG"].iloc[0]
    X = data_past[["SEMANA"]]
    y = data_past["US/KG"]
    y = y.fillna(y.mean())
    prediccion = fun_predict(X, y, 52)
    #print(data_predict - prediccion)
    return data_predict - prediccion       # Funcional programming


In [7]:
def predict_rolling(X, y, semana):
    rollin = y.rolling(window=3).mean()
    return rollin.iloc[-1]

In [8]:
paises = entradallm["PAIS DESTINO"].unique()
errores = [eval_pais(pais, predict_rolling) for pais in paises]   #list comprehension, es más rápido que un for
errores = np.array([e for e in errores if e is not None])
np.mean(errores**2)

np.float64(0.26796757137439864)

In [ ]:
plt.plot(germany["SEMANA"],germany["US/KG"].rolling(window=3).mean())
plt.scatter(52, germany[germany["SEMANA"]==52]["US/KG"],  color="red")
plt.show()

In [ ]:
X = data_past[["SEMANA"]]
y = data_past["US/KG"].rolling(window=3).mean()
y = y.fillna(y.mean())

In [12]:
# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X, y)

# Coeficientes de la regresión
slope = model.coef_[0]  # Pendiente
intercept = model.intercept_  # Intersección

print(f"Pendiente (slope): {slope}")
print(f"Intersección (intercept): {intercept}")

# Predicciones
prediccion = model.predict(germany[["SEMANA"]])

# Visualización de los datos y la línea de regresión
plt.scatter(germany["SEMANA"], germany["US/KG"], color='blue', label='Datos reales')
plt.plot(germany["SEMANA"], prediccion, color='red', label='Línea de regresión')
plt.xlabel('Semana')
plt.ylabel('US/kG')
plt.title('Regresión Lineal: Semana vs US/kG')
plt.legend()
plt.show()

NameError: name 'X' is not defined

In [13]:
#Actualizado